# A Comparison Analysis of CAO points for 2019, 2020, 2021
Source: [CAO Webpage](http://www.cao.ie/index.php?page=mediastats)
***


## Import Libraries

***



In [862]:
# Imports libraries
import pandas as pd

# Create plots.
import matplotlib.pyplot as plt

# Nice plot style. 
import seaborn as sns

# Numerical operations. 
import numpy as np

# Regular expressions
import re

# HTTP requests
import requests as rq

# Dates and times
import datetime as dt

# Opening URLs 
import urllib.request as urlrq

import openpyxl as oxl

# Ensures plots are shown.
%matplotlib inline

## Funtion to retrieve current date and time

In [863]:
def time():
    # Gets the current date and time
    cur_time = dt.datetime.now()
    # Format as a string
    current_time = cur_time.strftime('%Y%m%d_%H%M%S')
    return current_time

<br>

# 2021 CAO data

http://www.cao.ie/index.php?page=points&p=2021&bb=points

***



## Retrieve data from webserver

In [864]:
# Retrieves CAO points from the webserver.
response = rq.get('http://www2.cao.ie/points/l8.php')

# Response 200 signifies a successful request/response.
response

<Response [200]>

<br>

## Save original data

In [865]:
# Creates a file path for the original data
pathHTML = 'data/cao2021' + time() + '.html'# Note the importance of the filename and how it
                                              # will be easy to find in folders/sorted appropriately

In [866]:
# Saves the original html file.
with open(pathHTML, 'w') as f:
    f.write(response.text)

<br>

## Charset error on server

***


Technically, server states decoding as:

```
    Content-Type: text/html; charset=iso-8859-1.
``` 

However, one line uses \x96 which isn't defined in iso-8859-1. 

Therefore, we use the similar decoding standard cp1252, which is very similar but includes \x96. 

In [867]:
# Server uses incorrect encoding
orig_encoding = response.encoding

# Corrected encoding to cp1252
response.encoding = 'cp1252'

<br>

## Using regular expressions to extract desired data
***

To do: Explain what the regular expression is doing. COnsider doing step-by-step explanation. 

In [868]:
# Compiles the regular expression for matching lines so it doesn't recompile repeatedly.
re_course = re.compile(r'([A-Z]{2}[0-9]{3})  (.*)')  # 'r' python treats string as raw string and doesnt evaluate back slashes
                                                                    # \ {character} means we want the literal character ie., *
                                                                    # ? means 0 or 1 of 
                                                                    # + means 1 or more of 

<br>

## Iterating through the response line by line
***

To do:

Here we are dealing with the hash and astrisks denoting portfolio and random selection consecutively.

make sure to spell out what hash and * mean. 

Include image as example. 

In [869]:
# Function to separate the *, # and digits in the points string. 

def points_to_list(string):
    portfolio = ''
    if string[0] == '#':
        portfolio = '#'
    random = ''
    if string[-1] == "*":
        random = '*'
    points = ''
    for i in string:
        # Extracts only the digits from the string. 
        if i.isdigit():
            points += i
    return [points, portfolio, random]

In [870]:
# Path for CSV file
path2021 = 'data/cao2021' + time() + '.csv'

# Keeping count of the courses we are processing.
course_count = 0

# Opens the CSV file for wrting.
with open(path2021, 'w') as f:
    # Loops through and prints data from response line by line.
    for line in response.iter_lines():
        # Decoding turns bytes into code points and those code points can be displayed on the screen
        d_line = line.decode('cp1252')
        # Match the string specified in re_course, returning only the courses from the response
        if re_course.fullmatch(d_line):
            # Adds one to the course count
            course_count += 1
            # Extract course code and strip of any white space.
            course_code = d_line[:5].strip()
            # Extract course title and strip it of any white space.
            course_title = d_line[7:57].strip()
            # Points.
            course_points = re.split(' +' , d_line[60:])
            # The last course created 3 substrings w/ line split above & the last substring was subsequently removed.
            # Removing substrings that are not useful. 
            if len(course_points) != 2:
                course_points = course_points[:2]
            # Rejoin the substrings with commas. Because course points is a list we need to specify both items. 
            line_split = [course_code, course_title, course_points[0], course_points[1]]
            f.write(','.join(line_split) + '\n')

            # https://web.microsoftstream.com/video/89e78275-d944-444e-bed7-d964a3bd2c35 30.00


After writing the data to file, I went into the CSV and created a row with headings for each column. 

In [871]:
# Read in 2021 csv file and add header row.
df_2021 = pd.read_csv("data/cao202120211115_190905.csv", 
                names=["course_code", "course_title", "rnd_1", "rnd_2"])    

df_2021['rnd_1'] = df['rnd_1'].to_numpy()

df_2021

,course_code,course_title,rnd_1,rnd_2
0,AL801,Software Design for Virtual Reality and Gaming...,300,NaN
1,AL802,Software Design in Artificial Intelligence for...,313,NaN
2,AL803,Software Design for Mobile Apps and Connected ...,350,NaN
3,AL805,Computer Engineering for Network Infrastructur...,321,NaN
4,AL810,Quantity Surveying ...,328,NaN
...,...,...,...,...
944,WD211,Creative Computing ...,270,NaN
945,WD212,Recreation and Sport Management ...,262,NaN
946,WD230,Mechanical and Manufacturing Engineering ...,230,230
947,WD231,Early Childhood Care and Education ...,266,NaN


On the 12-11-2021, there was verification that 949 courses were documented in the 2021 CAO data online at the following webpage http://www2.cao.ie/points/l8.php. This corresponds with the number of courses in our dataset. 
***


## 2020 CAO data

http://www.cao.ie/index.php?page=points&p=2020
***

In [872]:
url_2020 = 'http://www2.cao.ie/points/CAOPointsCharts2020.xlsx'

### Save original file
***

In [873]:
# Create a file path for the original data set.
pathxlsx = 'data/CAO2020' + time() + '.xlsx'

# Opening URL
urlrq.urlretrieve(url_2020, pathxlsx)

('data/CAO202020211125_110514.xlsx', <http.client.HTTPMessage at 0x1148ad2e0>)


## Load data with pandas
https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html
***

changed file from xlxs to xls as in PP (save as) as xldr does not support files ending in anything other than xls. 

In [874]:
# Load and parse spread sheet. 
df_2020 = pd.read_excel(pathxlsx, skiprows=10, engine='openpyxl')# Use openpyxl to open xlsx spreadsheet.
df_2020

,CATEGORY (i.e.ISCED description),COURSE TITLE,COURSE CODE2,R1 POINTS,R1 Random *,R2 POINTS,R2 Random*,EOS,EOS Random *,EOS Mid-point,...,avp,v,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
0,Business and administration,International Business,AC120,209,NaN,NaN,NaN,209,NaN,280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Humanities (except languages),Liberal Arts,AC137,252,NaN,NaN,NaN,252,NaN,270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arts,"First Year Art & Design (Common Entry,portfolio)",AD101,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arts,Graphic Design and Moving Image Design (portfo...,AD102,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arts,Textile & Surface Design and Jewellery & Objec...,AD103,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459,Manufacturing and processing,Manufacturing Engineering,WD208,188,NaN,NaN,NaN,188,NaN,339,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1460,Information and Communication Technologies (ICTs),Software Systems Development,WD210,279,NaN,NaN,NaN,279,NaN,337,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1461,Information and Communication Technologies (ICTs),Creative Computing,WD211,271,NaN,NaN,NaN,271,NaN,318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1462,Personal services,Recreation and Sport Management,WD212,270,NaN,NaN,NaN,270,NaN,349,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [875]:
# Delete irrelevant columns
# df_20.drop(columns=['COURSE TITLE', 
#                     'CATEGORY (i.e.ISCED description)', 
#                     'R1 Random *', 
#                     'R2 Random*', 
#                     'EOS', 
#                     'EOS Random *', 
#                     'EOS Mid-point', 
#                     'HEI', 
#                     'avp', 
#                     'v', 
#                     'Column1', 'Column2', 'Column3', 'Column4', 'Column5', 'Column6', 
#                     'Column7', 'Column8'])


In [876]:
# Checking random row to ensure data integrity.
df_2020.iloc[650]

CATEGORY (i.e.ISCED description)                                             Arts
COURSE TITLE                        Arts (Drama, Theatre and Performance Studies)
COURSE CODE2                                                                GY118
R1 POINTS                                                                     451
R1 Random *                                                                   NaN
R2 POINTS                                                                     NaN
R2 Random*                                                                    NaN
EOS                                                                           451
EOS Random *                                                                  NaN
EOS Mid-point                                                                 492
LEVEL                                                                           8
HEI                                        National University of Ireland, Galway
Test/Interview #

In [877]:
# Another check
df_2020.iloc[-1]


CATEGORY (i.e.ISCED description)          Engineering and engineering trades
COURSE TITLE                        Mechanical and Manufacturing Engineering
COURSE CODE2                                                           WD230
R1 POINTS                                                                253
R1 Random *                                                              NaN
R2 POINTS                                                                NaN
R2 Random*                                                               NaN
EOS                                                                      253
EOS Random *                                                             NaN
EOS Mid-point                                                            369
LEVEL                                                                      8
HEI                                        Waterford Institute of Technology
Test/Interview #                                                         NaN

In [878]:
# Saving data to CSV file.
path2020 = 'data/cao2020_' + time() + '.csv'
#df_2020.to_csv(path2020, names=["area", "course_title", "course_code", ])

df_2020

,CATEGORY (i.e.ISCED description),COURSE TITLE,COURSE CODE2,R1 POINTS,R1 Random *,R2 POINTS,R2 Random*,EOS,EOS Random *,EOS Mid-point,...,avp,v,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
0,Business and administration,International Business,AC120,209,NaN,NaN,NaN,209,NaN,280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Humanities (except languages),Liberal Arts,AC137,252,NaN,NaN,NaN,252,NaN,270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arts,"First Year Art & Design (Common Entry,portfolio)",AD101,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arts,Graphic Design and Moving Image Design (portfo...,AD102,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arts,Textile & Surface Design and Jewellery & Objec...,AD103,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459,Manufacturing and processing,Manufacturing Engineering,WD208,188,NaN,NaN,NaN,188,NaN,339,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1460,Information and Communication Technologies (ICTs),Software Systems Development,WD210,279,NaN,NaN,NaN,279,NaN,337,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1461,Information and Communication Technologies (ICTs),Creative Computing,WD211,271,NaN,NaN,NaN,271,NaN,318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1462,Personal services,Recreation and Sport Management,WD212,270,NaN,NaN,NaN,270,NaN,349,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<br>

# 2019 CAO data
 
http://www2.cao.ie/points/lvl8_19.pdf

***


## Convert pdf to csv
***

What did I do to prepare data before reading in csv file below?

Copy and pasted pdf in Preview, pasted into a Word document so it formats nicely.

Then copied the data from the Word document to a csv file, while deleting preamble and unnecessary data such as preamble, page numbers and full Higher Education INstitution's names while keeping the course code and points etc. 

In [879]:
# Read in 2019 cao csv file & use the tab character as the delimiter.
df_2019 = pd.read_csv('data/cao2019_20211101_213010.csv', sep='\t')
df_2019

,Course Code,INSTITUTION and COURSE,EOS,Mid
0,AL801,Software Design with Virtual Reality and Gaming,304,328
1,AL802,Software Design with Cloud Computing,301,306
2,AL803,Software Design with Mobile Apps and Connected...,309,337
3,AL805,Network Management and Cloud Infrastructure,329,442
4,AL810,Quantity Surveying,307,349
...,...,...,...,...
925,WD200,Arts (options),221,296
926,WD210,Software Systems Development,271,329
927,WD211,Creative Computing,275,322
928,WD212,Recreation and Sport Management,274,311


In [880]:
# Check columns to find white space. 
df_2019.columns

Index(['Course Code ', 'INSTITUTION and COURSE ', 'EOS ', 'Mid '], dtype='object')

In [881]:
# Remove white space in column titles.
df_2019.columns = df_2019.columns.str.strip()

As seen above there is white space at the end of the course code and course title strings. It is important to move any white space now as it will impede access to the data later on. 


In [882]:
# Strip data of any white space.
df_2019['Course Code'] = df_2019['Course Code'].str.strip()
df_2019['INSTITUTION and COURSE'] = df_2019['INSTITUTION and COURSE'].str.strip()

## Concatenate & Join Data Sets
***

In [883]:
courses2021 = df_2021[['course_code', 'course_title']]

In [884]:
courses2021

,course_code,course_title
0,AL801,Software Design for Virtual Reality and Gaming...
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructur...
4,AL810,Quantity Surveying ...
...,...,...
944,WD211,Creative Computing ...
945,WD212,Recreation and Sport Management ...
946,WD230,Mechanical and Manufacturing Engineering ...
947,WD231,Early Childhood Care and Education ...


In [885]:
# Create new data frame with columns that are applicable to our analysis.
courses2020 = df_2020[['COURSE CODE2', 'COURSE TITLE']]
courses2020.columns = ['course_code', 'course_title']

In [886]:
courses2020

,course_code,course_title
0,AC120,International Business
1,AC137,Liberal Arts
2,AD101,"First Year Art & Design (Common Entry,portfolio)"
3,AD102,Graphic Design and Moving Image Design (portfo...
4,AD103,Textile & Surface Design and Jewellery & Objec...
...,...,...
1459,WD208,Manufacturing Engineering
1460,WD210,Software Systems Development
1461,WD211,Creative Computing
1462,WD212,Recreation and Sport Management


In [887]:
df_2019.columns

Index(['Course Code', 'INSTITUTION and COURSE', 'EOS', 'Mid'], dtype='object')

In [888]:
courses2019 = df_2019[['Course Code', 'INSTITUTION and COURSE', ]]
courses2019.columns = ['course_code', 'course_title', ]
courses2019

,course_code,course_title
0,AL801,Software Design with Virtual Reality and Gaming
1,AL802,Software Design with Cloud Computing
2,AL803,Software Design with Mobile Apps and Connected...
3,AL805,Network Management and Cloud Infrastructure
4,AL810,Quantity Surveying
...,...,...
925,WD200,Arts (options)
926,WD210,Software Systems Development
927,WD211,Creative Computing
928,WD212,Recreation and Sport Management


#### Concatenate Data Frames
***

Pandas function to concatenate data frames: 

[Documentation](https://pandas.pydata.org/docs/reference/api/pandas.concat.html) 

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
`pandas.concat(objs, axis=0, join='outer', ignore_index=False, keys=None, levels=None,` 

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;`names=None, verify_integrity=False, sort=False, copy=True)`&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;

Setting `ignore_index` to True ensures that the indices from the old data frames are not brought into the new data frame. This is to ensure no duplication of indices. For example, we do not want multiple rows all with 0 as their index.

In [889]:
# Concatenating 2021 & 2021 courses. 
all_courses = pd.concat([courses2021, courses2020, courses2019], ignore_index=True)
all_courses

,course_code,course_title
0,AL801,Software Design for Virtual Reality and Gaming...
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructur...
4,AL810,Quantity Surveying ...
...,...,...
3338,WD200,Arts (options)
3339,WD210,Software Systems Development
3340,WD211,Creative Computing
3341,WD212,Recreation and Sport Management


In [890]:
# Reset index.
all_courses.reset_index()

,index,course_code,course_title
0,0,AL801,Software Design for Virtual Reality and Gaming...
1,1,AL802,Software Design in Artificial Intelligence for...
2,2,AL803,Software Design for Mobile Apps and Connected ...
3,3,AL805,Computer Engineering for Network Infrastructur...
4,4,AL810,Quantity Surveying ...
...,...,...,...
3338,3338,WD200,Arts (options)
3339,3339,WD210,Software Systems Development
3340,3340,WD211,Creative Computing
3341,3341,WD212,Recreation and Sport Management


## Manage Duplicate Rows
***

To deal with duplicate rows, the following functions form the pandas library can be used: 

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;`pandas.DataFrame.duplicated()` [Documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.duplicated.html)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;`pandas.DataFrame.drop_duplicates()` [Documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html)

In [891]:
# Find extra duplicates based on course code. 
all_courses[all_courses.duplicated(subset=['course_code'])]

,course_code,course_title
949,AC120,International Business
950,AC137,Liberal Arts
951,AD101,"First Year Art & Design (Common Entry,portfolio)"
952,AD102,Graphic Design and Moving Image Design (portfo...
953,AD103,Textile & Surface Design and Jewellery & Objec...
...,...,...
3338,WD200,Arts (options)
3339,WD210,Software Systems Development
3340,WD211,Creative Computing
3341,WD212,Recreation and Sport Management


In [892]:
# Remove duplicate rows, implementing changes in-place, while ignoring indices. 
all_courses.drop_duplicates(subset=['course_code'], inplace=True, ignore_index=True)

In [893]:
# Take a look at the data. 
all_courses

,course_code,course_title
0,AL801,Software Design for Virtual Reality and Gaming...
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructur...
4,AL810,Quantity Surveying ...
...,...,...
1646,SG441,Environmental Science
1647,SG446,Applied Archaeology
1648,TL803,Music Technology
1649,TL812,Computing with Digital Media


In [894]:
# Compare two rows.
all_courses.loc[175] == all_courses.loc[176]

course_code     False
course_title    False
dtype: bool

<br>

### Join the points
***

This code joins the 2021 points to the all_course dataframe. 

In [895]:
df_2021

,course_code,course_title,rnd_1,rnd_2
0,AL801,Software Design for Virtual Reality and Gaming...,300,NaN
1,AL802,Software Design in Artificial Intelligence for...,313,NaN
2,AL803,Software Design for Mobile Apps and Connected ...,350,NaN
3,AL805,Computer Engineering for Network Infrastructur...,321,NaN
4,AL810,Quantity Surveying ...,328,NaN
...,...,...,...,...
944,WD211,Creative Computing ...,270,NaN
945,WD212,Recreation and Sport Management ...,262,NaN
946,WD230,Mechanical and Manufacturing Engineering ...,230,230
947,WD231,Early Childhood Care and Education ...,266,NaN


In [896]:
# Set index to be the course_code column and save in-place. 
df_2021.set_index('course_code', inplace=True)

# Change df_2021 column names to include the year.
df_2021.columns = ['course_title', 'points_r1_2021', 'points_r2_2021']

# Check dataframe.
df_2021

,course_title,points_r1_2021,points_r2_2021
course_code,,,
AL801,Software Design for Virtual Reality and Gaming...,300,NaN
AL802,Software Design in Artificial Intelligence for...,313,NaN
AL803,Software Design for Mobile Apps and Connected ...,350,NaN
AL805,Computer Engineering for Network Infrastructur...,321,NaN
AL810,Quantity Surveying ...,328,NaN
...,...,...,...
WD211,Creative Computing ...,270,NaN
WD212,Recreation and Sport Management ...,262,NaN
WD230,Mechanical and Manufacturing Engineering ...,230,230


In [897]:
# Set the all_courses index to be the course_code column.
all_courses.set_index('course_code', inplace=True)

In [898]:
# Join the 2021 points to the all_courses dataframe.
all_courses = all_courses.join(df_2021[['points_r1_2021', 'points_r1_2021']])
all_courses

,course_title,points_r1_2021,points_r1_2021
course_code,,,
AL801,Software Design for Virtual Reality and Gaming...,300,300
AL802,Software Design in Artificial Intelligence for...,313,313
AL803,Software Design for Mobile Apps and Connected ...,350,350
AL805,Computer Engineering for Network Infrastructur...,321,321
AL810,Quantity Surveying ...,328,328
...,...,...,...
SG441,Environmental Science,NaN,NaN
SG446,Applied Archaeology,NaN,NaN
TL803,Music Technology,NaN,NaN


Join the 2020 points to the all_courses dataframe.

In [899]:
df_2020.columns

Index(['CATEGORY (i.e.ISCED description)', 'COURSE TITLE', 'COURSE CODE2',
       'R1 POINTS', 'R1 Random *', 'R2 POINTS', 'R2 Random*', 'EOS',
       'EOS Random *', 'EOS Mid-point', 'LEVEL', 'HEI', 'Test/Interview #',
       'avp', 'v', 'Column1', 'Column2', 'Column3', 'Column4', 'Column5',
       'Column6', 'Column7', 'Column8'],
      dtype='object')

In [900]:
# Extract columns. 
df_2020 = df_2020[['COURSE CODE2', 'R1 POINTS', 'R2 POINTS']]

# Change column names.
df_2020.columns = ['course_code', 'points_r1_2020', 'points_r2_2020']
df_2020

,course_code,points_r1_2020,points_r2_2020
0,AC120,209,NaN
1,AC137,252,NaN
2,AD101,#+matric,NaN
3,AD102,#+matric,NaN
4,AD103,#+matric,NaN
...,...,...,...
1459,WD208,188,NaN
1460,WD210,279,NaN
1461,WD211,271,NaN
1462,WD212,270,NaN


In [901]:
# Set index to be course_code column. 
df_2020.set_index('course_code', inplace=True)

In [902]:
# Join 2020 points to all_courses dataframe.
all_courses = all_courses.join(df_2020)

# Check that the points were added.
all_courses

,course_title,points_r1_2021,points_r1_2021,points_r1_2020,points_r2_2020
course_code,,,,,
AL801,Software Design for Virtual Reality and Gaming...,300,300,303,NaN
AL802,Software Design in Artificial Intelligence for...,313,313,332,NaN
AL803,Software Design for Mobile Apps and Connected ...,350,350,337,NaN
AL805,Computer Engineering for Network Infrastructur...,321,321,333,NaN
AL810,Quantity Surveying ...,328,328,319,NaN
...,...,...,...,...,...
SG441,Environmental Science,NaN,NaN,NaN,NaN
SG446,Applied Archaeology,NaN,NaN,NaN,NaN
TL803,Music Technology,NaN,NaN,NaN,NaN


In [903]:
df_2019.columns

Index(['Course Code', 'INSTITUTION and COURSE', 'EOS', 'Mid'], dtype='object')

In [904]:
# Extract columns. 
df_2019 = df_2019[['Course Code', 'Mid', 'EOS']]

# Change column names.
df_2019.columns = ['course_code', 'mid_2019', 'eos_2019']
df_2019

,course_code,mid_2019,eos_2019
0,AL801,328,304
1,AL802,306,301
2,AL803,337,309
3,AL805,442,329
4,AL810,349,307
...,...,...,...
925,WD200,296,221
926,WD210,329,271
927,WD211,322,275
928,WD212,311,274


In [905]:
df_2019.set_index('course_code', inplace=True)

In [906]:
# Join 2019 points to all_courses dataframe.
all_courses = all_courses.join(df_2019)

# Check.
all_courses

,course_title,points_r1_2021,points_r1_2021,points_r1_2020,points_r2_2020,mid_2019,eos_2019
course_code,,,,,,,
AL801,Software Design for Virtual Reality and Gaming...,300,300,303,NaN,328,304
AL802,Software Design in Artificial Intelligence for...,313,313,332,NaN,306,301
AL803,Software Design for Mobile Apps and Connected ...,350,350,337,NaN,337,309
AL805,Computer Engineering for Network Infrastructur...,321,321,333,NaN,442,329
AL810,Quantity Surveying ...,328,328,319,NaN,349,307
...,...,...,...,...,...,...,...
SG441,Environmental Science,NaN,NaN,NaN,NaN,358,297
SG446,Applied Archaeology,NaN,NaN,NaN,NaN,290,289
TL803,Music Technology,NaN,NaN,NaN,NaN,288,264


In [907]:
df_2019['mid_2019']

course_code
AL801    328 
AL802    306 
AL803    337 
AL805    442 
AL810    349 
         ... 
WD200    296 
WD210    329 
WD211    322 
WD212    311 
WD230    348 
Name: mid_2019, Length: 930, dtype: object

***
# End